In [ ]:
!pip install -q datasets transformers[torch] accelerate evaluate shap sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("iNeil77/CodeNet", "C++")
print(ds)

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['s_id', 'p_id', 'u_id', 'date', 'language', 'original_language', 'filename_ext', 'status', 'cpu_time', 'memory', 'code_size', 'code'],
        num_rows: 7982624
    })
})


In [ ]:
from collections import Counter

# Define a simple function to assign labels
def add_labels(example):
    # 0 if status is "Accepted", else 1
    example["bug_label"] = 0 if example["status"] == "Accepted" else 1
    return example

# Apply the function only to the "train" split
ds["train"] = ds["train"].map(add_labels)

# Check the distribution of 'bug_label'
print("Bug label distribution:", Counter(ds["train"]["bug_label"]))


Map:   0%|          | 0/7982624 [00:00<?, ? examples/s]

Bug label distribution: Counter({0: 4341215, 1: 3641409})
